# 03 - Recommender System

In [1]:
# imports
import pandas as pd
import numpy as np
import time
import pickle
import sqlite3

from sqlalchemy import create_engine
from scipy import sparse # cut down on memory size
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

pd.options.display.max_columns = 35

___

In [2]:
# read in the data
df = pd.read_csv('../Data/cards_cleaned.csv')
df.head()

,name,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,oracle_text_token,legalities,rarity,power,toughness,loyalty,activated_ability,triggered_ability,oracle_text_back,oracle_text_back_token,colors_back,power_back,toughness_back,loyalty_back,card_type_back,super_type_back,sub_type_back,mana_cost_back,scryfall_uri
0,Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,"As long as Static Orb is untapped, players can...",as long as static orb is untapped players can'...,legacy vintage commander duel,rare,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,https://scryfall.com/card/7ed/319/static-orb?u...
1,Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,Enchant creature Enchanted creature gets -3/-0.,enchant creature enchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,https://scryfall.com/card/m14/71/sensory-depri...
2,Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,Choose one — • Return target permanent card fr...,choose one return target permanent card from y...,legacy vintage commander duel,rare,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,https://scryfall.com/card/c19/34/road-of-retur...
3,Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,Flying (This creature can't be blocked except ...,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,1,2,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,https://scryfall.com/card/9ed/100/storm-crow?u...
4,Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{T}: Target creature loses your choice of flyi...,{t} target creature loses your choice of flyin...,legacy vintage penny commander duel,uncommon,1,1,NONE,1.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,https://scryfall.com/card/ulg/47/walking-spong...


In [3]:
df.shape

(20053, 30)

___

## Oracle text recommender system
To start I want to build a recommender system that just looks at oracle text. In order to do that I need to combine both oracle_text and oracle_text_back into one single column

In [4]:
df['oracle_combined'] = df['oracle_text_token'] + " " + df['oracle_text_back_token']

In [5]:
# start by isolating the name of the card and it's oracle_text
oracle = df['oracle_combined']

# vectorize all our words
cvec = CountVectorizer(stop_words=['none'],
                      min_df=2,
                      max_df=.99,
                      ngram_range=(1,6),
                      token_pattern="[a-zA-Z{}+'0-9-/−]+") # we should use the same RegEx to keep certain characters together 

oracle_vec = cvec.fit_transform(oracle)

# convert to a dataframe so we can use this later on as well
converted_df = pd.DataFrame(oracle_vec.toarray(), columns=cvec.get_feature_names(), index=df['name'])

In [6]:
converted_df.shape

(20053, 142203)

This is a whole lot of features to examine so I'm going to use Variance Threshold to drop some n_gram columns that only exist in a small number of cards

In [7]:
selector = VarianceThreshold(.001)
new_array = selector.fit_transform(converted_df)

new_array.shape

(20053, 14862)

In [8]:
# get the list of columns kept
vt_list = converted_df.columns[selector.get_support()]

# update our converted_df with just the important columns
converted_df = converted_df[vt_list]

For the recommender system to work efficiently we need to convert the data back into a sparse matrix with the new names as the index

In [9]:
sparse_df = sparse.csr_matrix(converted_df)

In [10]:
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

In [11]:
rec.shape

(20053, 20053)

In [12]:
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Stinging Lionfish,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.00000,1.000000,1.000000,0.976560,1.000000,0.915485,0.975803,1.000000,1.000000,1.000000,1.000000,0.895652,0.979257,0.932116,0.982300,0.961689,1.000000,...,0.987429,0.797163,0.919008,0.983164,0.974746,0.979739,1.000000,1.000000,0.921379,0.911501,1.000000,1.000000,1.0,0.985243,1.000000,0.943341,1.000000
Sensory Deprivation,1.00000,0.000000,1.000000,0.939834,0.921311,1.000000,1.000000,0.840510,0.952886,1.000000,1.000000,0.625030,0.946757,1.000000,0.954569,1.000000,1.000000,...,1.000000,0.869842,1.000000,0.956786,1.000000,0.895990,1.000000,0.943612,1.000000,1.000000,1.000000,1.000000,1.0,0.962122,0.947995,0.927285,1.000000
Road of Return,1.00000,1.000000,0.000000,1.000000,0.925646,1.000000,0.917840,0.984930,0.910962,1.000000,0.952836,0.979754,0.979876,0.884749,0.793944,0.900888,0.870359,...,0.859751,0.909809,0.895234,0.942834,0.853001,0.970516,0.801196,0.925405,0.974575,0.948486,0.911906,0.695338,1.0,0.928418,0.950860,0.855710,1.000000
Storm Crow,0.97656,0.939834,1.000000,0.000000,0.919516,1.000000,1.000000,0.979609,0.927716,1.000000,1.000000,0.958908,0.959156,0.955445,0.976766,0.907804,1.000000,...,0.991749,0.789214,0.982281,0.966850,0.983425,0.946808,1.000000,1.000000,0.982799,0.988383,0.981662,1.000000,1.0,0.970944,0.960106,0.972110,0.962602
Walking Sponge,1.00000,0.921311,0.925646,0.919516,0.000000,1.000000,0.937689,0.546626,0.952731,0.906341,1.000000,0.946257,0.928776,0.970864,0.848066,0.791724,0.971323,...,0.913674,0.941962,0.884128,0.956645,0.913289,0.930432,0.960910,0.886853,1.000000,1.000000,0.916058,0.960064,1.0,0.809990,0.913040,0.768979,1.000000


In [13]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Unyaro Bee Sting     0.000000
Tarfire              0.000000
Shock                0.000000
Bee Sting            0.000000
Magma Jet            0.058487
Deadapult            0.118083
Moonglove Extract    0.133975
Shock Troops         0.133975
Crackling Triton     0.149037
Seismic Assault      0.149037
Goblin Test Pilot    0.149037
Name: Shock, dtype: float64

In [14]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb                0.000000
Winter Orb                0.102915
Damping Field             0.349668
Imi Statue                0.349668
Smoke                     0.395633
Stoic Angel               0.433053
Storage Matrix            0.532536
Castle Raptors            0.535165
Kill Switch               0.570685
Giant Tortoise            0.587486
Juniper Order Advocate    0.613976
Name: Static Orb, dtype: float64

In [15]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Reassembling Skeleton     0.374905
Footsteps of the Goryo    0.378368
Bone Dragon               0.383870
Chronosavant              0.408758
Scrapheap Scrounger       0.409055
Apprentice Necromancer    0.414345
Despoiler of Souls        0.418516
Wake the Dead             0.423640
Ghoulsteed                0.424205
Cauldron Dance            0.425656
Name: Prized Amalgam, dtype: float64

In [16]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Wrath of God               0.000000
Damnation                  0.000000
Perish                     0.117647
Winds of Rath              0.172660
Shatterstorm               0.185908
Plague Wind                0.207882
Abu Ja'far                 0.238958
Jokulhaups                 0.240743
Do or Die                  0.341689
Retribution of the Meek    0.344064
Obliterate                 0.356079
Name: Wrath of God, dtype: float64

In [17]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Grizzly Bears                 0.0
Static Orb                    1.0
Mask of Avacyn                1.0
Saheeli, Sublime Artificer    1.0
Saheeli's Silverwing          1.0
Dungeon Geists                1.0
Grind // Dust                 1.0
Rage Nimbus                   1.0
Fiery Hellhound               1.0
The Antiquities War           1.0
Jace's Phantasm               1.0
Name: Grizzly Bears, dtype: float64

This is a great start! Now I want to add the numerical features and see the results
___

## Adding numerical data to our features

In [18]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back             object
scryfall_uri  

In [19]:
# we already have a df for the oracle text so we can just use that one and add to it.
converted_df['cmc'] = df['cmc'].values
converted_df['activated_ability'] = df['activated_ability'].values
converted_df['triggered_ability'] = df['triggered_ability'].values

# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Stinging Lionfish,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.883752,0.911060,0.861461,0.849576,0.533628,0.716358,0.847796,0.952542,0.589635,0.842051,0.760043,0.829526,0.582439,0.852365,0.933497,0.771694,...,0.862899,0.702867,0.798653,0.821901,0.749861,0.879364,0.884814,0.753402,0.802281,0.755819,0.829424,0.742903,0.506803,0.911477,0.734258,0.811634,0.745314
Sensory Deprivation,0.883752,0.000000,0.957495,0.886499,0.856223,0.794262,0.872918,0.781782,0.931959,0.803884,0.924515,0.582248,0.877794,0.817797,0.894167,0.984109,0.890891,...,0.939519,0.817426,0.935850,0.882149,0.889653,0.851750,0.944952,0.835008,0.937006,0.917626,0.918481,0.877132,0.764298,0.927476,0.830666,0.864968,0.878284
Road of Return,0.911060,0.957495,0.000000,0.956581,0.876247,0.842592,0.834710,0.930435,0.895884,0.849953,0.898932,0.924796,0.925201,0.790897,0.757083,0.890577,0.805217,...,0.820689,0.868074,0.852758,0.887291,0.788935,0.933836,0.768357,0.846716,0.927706,0.889709,0.856549,0.652179,0.819666,0.902897,0.862347,0.812747,0.906876
Storm Crow,0.861461,0.886499,0.956581,0.000000,0.853131,0.789837,0.870185,0.907120,0.907327,0.799666,0.922891,0.887040,0.887649,0.787290,0.913512,0.894487,0.888544,...,0.930495,0.740971,0.918088,0.889646,0.873189,0.899041,0.943768,0.879614,0.919564,0.905337,0.900073,0.874489,0.759228,0.935177,0.837835,0.905169,0.844583
Walking Sponge,0.849576,0.856223,0.876247,0.853131,0.000000,0.733777,0.786224,0.482317,0.911955,0.661636,0.902323,0.856908,0.826053,0.730550,0.767189,0.784090,0.835283,...,0.843475,0.868750,0.813228,0.847501,0.785817,0.872110,0.893152,0.756002,0.918486,0.893409,0.820675,0.809211,0.695003,0.765387,0.767189,0.694222,0.842500


In [20]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Shock                     0.000000e+00
Tarfire                   4.440892e-16
Magma Jet                 6.580127e-02
Bee Sting                 1.237505e-01
Unyaro Bee Sting          1.237505e-01
Deadapult                 1.588004e-01
Moonglove Extract         1.699426e-01
Weaponize the Monsters    1.708438e-01
Seal of Fire              1.708438e-01
Orcish Vandal             1.711375e-01
Ember Hauler              1.711375e-01
Name: Shock, dtype: float64

In [21]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb          0.000000
Winter Orb          0.097786
Damping Field       0.254356
Imi Statue          0.254356
Stoic Angel         0.306780
Smoke               0.322933
Castle Raptors      0.362814
Wardscale Dragon    0.415397
Skyrider Trainee    0.429966
Storage Matrix      0.441783
Mungha Wurm         0.447177
Name: Static Orb, dtype: float64

In [22]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Footsteps of the Goryo    0.352702
Reassembling Skeleton     0.362452
Bone Dragon               0.380206
Scrapheap Scrounger       0.392484
Apprentice Necromancer    0.397741
Cauldron Dance            0.398335
Despoiler of Souls        0.401777
False Memories            0.402052
Wake the Dead             0.403900
Stitchwing Skaab          0.424360
Name: Prized Amalgam, dtype: float64

In [23]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Damnation          0.000000
Wrath of God       0.000000
Perish             0.078235
Winds of Rath      0.085894
Shatterstorm       0.093307
Jokulhaups         0.116040
Plague Wind        0.116612
Obliterate         0.167576
Catastrophe        0.182008
Day of Judgment    0.183503
Child of Alara     0.187596
Name: Wrath of God, dtype: float64

In [24]:
rec_df['Jace, the Mind Sculptor'].sort_values()[0:11]

name
Jace, the Mind Sculptor    0.000000
Coral Fighters             0.371142
Voyage's End               0.386285
Brainstorm                 0.415736
Riverwise Augur            0.418030
Select for Inspection      0.429563
Eye Spy                    0.432218
Cavalier of Gales          0.432581
Anchor to the Aether       0.436233
Precognition               0.439288
Dream Cache                0.447703
Name: Jace, the Mind Sculptor, dtype: float64

In [25]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Triton Shorethief                     0.0
Scarwood Goblins                      0.0
Canyon Minotaur                       0.0
Pearled Unicorn                       0.0
Headless Horseman                     0.0
Looming Altisaur                      0.0
Garenbrig Carver // Shield's Might    0.0
Wandering Ones                        0.0
Bastion Enforcer                      0.0
Gore Swine                            0.0
Streetbreaker Wurm                    0.0
Name: Grizzly Bears, dtype: float64

___

## Non-numerical data
Now I have to convert the non-numerical data into numerical data

### Power / Toughness / Loyalty

Through some outside reasearch on ScryFall and how other sources interpret * s in power and toughness, I'm going to impute the * s as 0 and * +1s as 1 for both power and toughness. Same is true for loyalty

In [26]:
# power
zero_power_index = df.loc[(df['power'] == '*')].index
zero_power_back_index = df.loc[(df['power_back'] == '*')].index
df.loc[zero_power_index, 'power'] = 0
df.loc[zero_power_back_index, 'power_back'] = 0

one_power_index = df.loc[(df['power'] == '1+*')].index
df.loc[one_power_index, 'power'] = 1

two_power_index = df.loc[(df['power'] == '2+*')].index
df.loc[two_power_index, 'power'] = 2

# toughness
zero_toughness_index = df.loc[(df['toughness'] == '*')].index
zero_toughness_back_index = df.loc[(df['toughness_back'] == '*')].index
df.loc[zero_toughness_index, 'toughness'] = 0
df.loc[zero_toughness_back_index, 'toughness_back'] = 0

one_toughness_index = df.loc[(df['toughness'] == '1+*')].index
df.loc[one_toughness_index, 'toughness'] = 1

two_toughness_index = df.loc[(df['toughness'] == '2+*')].index
df.loc[two_toughness_index, 'toughness'] = 2

seven_toughness_index = df.loc[(df['toughness'] == '7-*')].index
df.loc[seven_toughness_index, 'toughness'] = 7

# loyalty
zero_loyalty_index = df.loc[(df['loyalty'] == 'X')].index
zero_loyalty_back_index = df.loc[(df['loyalty_back'] == 'X')].index
df.loc[zero_loyalty_index, 'loyalty'] = 0
df.loc[zero_loyalty_back_index, 'loyalty_back'] = 0

In [27]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back             object
scryfall_uri  

In [28]:
# add the newly made numerical columns to the converted df
converted_df['power'] = df['power'].values
converted_df['power_back'] = df['power_back'].values
converted_df['toughness'] = df['toughness'].values
converted_df['toughness_back'] = df['toughness_back'].values
converted_df['loyalty'] = df['loyalty'].values
converted_df['loyalty_back'] = df['loyalty_back'].values

In [29]:
converted_df.head()

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+2 until,+0/+2 until end,+0/+2 until end of,+0/+2 until end of turn,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 counters,...,{w} {t},{w}{u}{b}{r}{g},{w}{w},{x},{x} {t},−1,−2,−3,−6,−7,−8,cmc,activated_ability,triggered_ability,power_back,toughness_back,loyalty_back
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,0.0,NONE,NONE,NONE
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,NONE,NONE,NONE
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0,0.0,NONE,NONE,NONE


___

In [30]:
# set the index of df to be the same as the converted_df; the name of the card
df = df.set_index(df['name']).drop(columns='name')

In [31]:
# essentially makeing dummy variables for non-numerical data

# initialize some lists to check
wburg = ['B', 'G', 'R', 'U', 'W']

card_types = ['Creature', 'Instant', 'Enchantment', 'Sorcery', 'Artifact', 'Land', 'Planeswalker', 'Tribal']

subtypes = list(set(" ".join(df['sub_type'].value_counts().index).split()))
subtypes.remove('NONE')

super_types = list(set(" ".join(df['super_type'].value_counts().index).split()))
super_types.remove('NONE')

formats = list(set(" ".join(df['legalities'].value_counts().index).split()))
formats.remove('NONE')

rarities = ['common', 'uncommon', 'rare', 'mythic']


# timer and progress checker
t0 = time.time()
counter = 0

for counter, index in enumerate(converted_df.index):
    # colors
    for color in wburg:
        converted_df.loc[index, 'colors_' + color] = (color in df.loc[index, 'colors']) * 1
        converted_df.loc[index, 'color_identity_' + color] = (color in df.loc[index, 'color_identity']) * 1
    
    # card_types
    for ctype in card_types:
        converted_df.loc[index, 'card_type_' + ctype] = (ctype in df.loc[index, 'card_type']) * 1
        
    # sub_types
    for stype in subtypes:
        converted_df.loc[index, 'sub_type_' + stype] = (stype in df.loc[index, 'sub_type']) * 1
        
    # super_type
    for st in super_types:
        converted_df.loc[index, 'super_type_' + st] = (st in df.loc[index, 'super_type']) * 1
    
    # legalities
    for form in formats:
        converted_df.loc[index, 'legalities_' + form] = (form in df.loc[index, 'legalities']) * 1
    
    # rarity
    for r in rarities:
        converted_df.loc[index, 'rarity_' + r] = (r in df.loc[index, 'rarity']) * 1
    
    # progress checker
    if counter % 1000 == 0:
        print(f'Converted {counter} cards out of a total of {len(converted_df.index)}')
        print(f'mins: {(time.time() - t0)/60}')
        print('-------------------------------')

Converted 0 cards out of a total of 20053
mins: 0.3687536319096883
-------------------------------
Converted 1000 cards out of a total of 20053
mins: 4.191409111022949
-------------------------------
Converted 2000 cards out of a total of 20053
mins: 7.877023164431254
-------------------------------
Converted 3000 cards out of a total of 20053
mins: 11.555680533250174
-------------------------------
Converted 4000 cards out of a total of 20053
mins: 15.258505213260651
-------------------------------
Converted 5000 cards out of a total of 20053
mins: 19.064158630371093
-------------------------------
Converted 6000 cards out of a total of 20053
mins: 23.289409931500753
-------------------------------
Converted 7000 cards out of a total of 20053
mins: 27.521571481227873
-------------------------------
Converted 8000 cards out of a total of 20053
mins: 31.27898144721985
-------------------------------
Converted 9000 cards out of a total of 20053
mins: 35.12320022980372
-------------------

In [32]:
# convert 'NONE's to np.NaN's so we can scale our data then impute all the NaNs
converted_df = converted_df.replace('NONE', np.NaN)

# convert those columns to numerical data
converted_df['power_back'] = converted_df['power_back'].astype(float)
converted_df['power'] = converted_df['power'].astype(float)
converted_df['toughness'] = converted_df['toughness'].astype(float)
converted_df['toughness_back'] = converted_df['toughness_back'].astype(float)
converted_df['loyalty'] = converted_df['loyalty'].astype(float)
converted_df['loyalty_back'] = converted_df['loyalty_back'].astype(float)

Now that we have everything coverted to numerical data, we should scale the non-binarized features so everything will be weighted equally. This means we should just scale the columns that have a range of values, e.g. power, toughness, loyalty, and cmc.

In [33]:
range_col_list = ['power', 'power_back', 'toughness', 'toughness_back', 'loyalty', 'loyalty_back', 'cmc']
converted_df[range_col_list]

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
name,,,,,,,
Static Orb,NaN,NaN,NaN,NaN,NaN,NaN,3.0
Sensory Deprivation,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Road of Return,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Storm Crow,1.0,NaN,2.0,NaN,NaN,NaN,2.0
Walking Sponge,1.0,NaN,1.0,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...
Devoted Hero,1.0,NaN,2.0,NaN,NaN,NaN,1.0
Without Weakness,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Firesong and Sunspeaker,4.0,NaN,6.0,NaN,NaN,NaN,6.0


In [34]:
ss = StandardScaler()
range_col_sc = ss.fit_transform(converted_df[range_col_list])

In [35]:
sc_df = pd.DataFrame(range_col_sc, index = df.index.values, columns=range_col_list)

In [36]:
sc_df.head()

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
Static Orb,NaN,NaN,NaN,NaN,NaN,NaN,-0.170451
Sensory Deprivation,NaN,NaN,NaN,NaN,NaN,NaN,-1.298523
Road of Return,NaN,NaN,NaN,NaN,NaN,NaN,-0.734487
Storm Crow,-0.902494,NaN,-0.425666,NaN,NaN,NaN,-0.734487
Walking Sponge,-0.902494,NaN,-1.007699,NaN,NaN,NaN,-0.734487


In [37]:
sc_df = sc_df.fillna(0)

In [38]:
sc_df.head()

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
Static Orb,0.000000,0.0,0.000000,0.0,0.0,0.0,-0.170451
Sensory Deprivation,0.000000,0.0,0.000000,0.0,0.0,0.0,-1.298523
Road of Return,0.000000,0.0,0.000000,0.0,0.0,0.0,-0.734487
Storm Crow,-0.902494,0.0,-0.425666,0.0,0.0,0.0,-0.734487
Walking Sponge,-0.902494,0.0,-1.007699,0.0,0.0,0.0,-0.734487


In [39]:
converted_df = pd.concat([converted_df.fillna(0).drop(columns=range_col_list), sc_df], axis=1)

In [40]:
# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

In [41]:
converted_df.isnull().sum().sum()

0

In [42]:
t0 = time.time()
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')
print((time.time() - t0)/60)

0.8578438520431518


In [43]:
# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Stinging Lionfish,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.871445,0.922203,0.900859,0.902281,0.773456,0.902228,0.901334,0.936167,0.827463,0.877416,0.842867,0.913040,0.836995,0.907935,0.918672,0.898334,...,0.946222,0.750877,0.853649,0.915079,0.874000,0.912737,0.946762,0.930249,0.820985,0.847184,0.927962,0.927541,0.804362,0.933875,0.923097,0.902166,0.856749
Sensory Deprivation,0.871445,0.000000,0.922075,0.740326,0.731101,0.743228,0.879411,0.703710,0.785529,0.768287,0.778069,0.509065,0.834759,0.916315,0.901549,0.910293,0.785016,...,0.915391,0.799558,0.745555,0.874081,0.901205,0.789081,0.901116,0.816191,0.886896,0.912487,0.939163,0.887339,0.630850,0.840174,0.898728,0.878727,0.713770
Road of Return,0.922203,0.922075,0.000000,0.954642,0.883275,0.887360,0.882046,0.906854,0.883198,0.864512,0.869790,0.944939,0.945291,0.867349,0.754494,0.866469,0.841961,...,0.838321,0.885705,0.866960,0.898628,0.816682,0.916103,0.773910,0.903310,0.902817,0.898988,0.889731,0.684946,0.881412,0.903104,0.925571,0.829844,0.918575
Storm Crow,0.900859,0.740326,0.954642,0.000000,0.769490,0.856457,0.916216,0.830234,0.818869,0.799273,0.830234,0.849643,0.885469,0.900968,0.918371,0.866902,0.861410,...,0.939718,0.743929,0.834325,0.918753,0.934143,0.856734,0.925272,0.924650,0.919663,0.942165,0.940720,0.938326,0.726982,0.901998,0.932736,0.937403,0.753520
Walking Sponge,0.902281,0.731101,0.883275,0.769490,0.000000,0.854997,0.864809,0.472315,0.795417,0.642079,0.800077,0.843931,0.828415,0.893923,0.786204,0.764381,0.852475,...,0.874250,0.860383,0.721176,0.905464,0.867994,0.816507,0.895198,0.819800,0.919379,0.941577,0.887611,0.905728,0.693766,0.765586,0.919032,0.764283,0.765721


In [44]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

name
Magma Jet                 0.094761
Tarfire                   0.107706
Weaponize the Monsters    0.154679
Ember Hauler              0.172696
Seal of Fire              0.220263
Explosive Apparatus       0.222512
Bee Sting                 0.237002
Unyaro Bee Sting          0.237002
Moonglove Extract         0.238159
Orcish Vandal             0.238199
Name: Shock, dtype: float64

In [45]:
rec_df['Lightning Bolt'].sort_values()[1:11]

name
Lightning Strike         0.072625
Searing Spear            0.075163
Volcanic Hammer          0.104106
Open Fire                0.116360
Ghostfire                0.131811
Fire Ambush              0.133049
Precision Bolt           0.137350
Valakut Invoker          0.181416
Fateful End              0.195262
Mudbutton Torchrunner    0.263199
Name: Lightning Bolt, dtype: float64

In [46]:
rec_df['Static Orb'].sort_values()[1:11]

name
Winter Orb        0.088955
Imi Statue        0.306607
Smoke             0.375713
Damping Field     0.394553
Stoic Angel       0.464872
Storage Matrix    0.475932
Kill Switch       0.502743
Stabilizer        0.558249
Castle Raptors    0.563905
Watchdog          0.576132
Name: Static Orb, dtype: float64

In [47]:
rec_df['Prized Amalgam'].sort_values()[1:11]

name
Footsteps of the Goryo    0.379850
Bone Dragon               0.405869
Scrapheap Scrounger       0.406195
Despoiler of Souls        0.412557
Wake the Dead             0.414103
Apprentice Necromancer    0.415079
False Memories            0.422891
Cauldron Dance            0.433453
Reassembling Skeleton     0.438698
Ghoulsteed                0.439678
Name: Prized Amalgam, dtype: float64

In [48]:
rec_df['Wrath of God'].sort_values()[1:11]

name
Damnation                  0.076467
Winds of Rath              0.142759
Perish                     0.236050
Day of Judgment            0.238798
Shatterstorm               0.253221
Retribution of the Meek    0.278155
Jokulhaups                 0.280239
Plague Wind                0.287375
Catastrophe                0.299175
Obliterate                 0.351529
Name: Wrath of God, dtype: float64

In [49]:
rec_df['Jace, the Mind Sculptor'].sort_values()[1:11]

name
Voyage's End             0.404479
Coral Fighters           0.405660
Select for Inspection    0.438152
Brainstorm               0.445944
Eye Spy                  0.458180
Anchor to the Aether     0.458734
Riverwise Augur          0.462491
Precognition             0.470662
Cavalier of Gales        0.476435
Dream Cache              0.482159
Name: Jace, the Mind Sculptor, dtype: float64

In [50]:
rec_df['Delver of Secrets // Insectile Aberration'].sort_values()[1:11]

name
Think Tank                               0.279352
Puresight Merrow                         0.324032
Geist of the Archives                    0.330405
Aberrant Researcher // Perfected Form    0.353065
Etherwrought Page                        0.358953
Galvanoth                                0.377329
Rummaging Wizard                         0.384179
Ink Dissolver                            0.391080
Into the Wilds                           0.399863
Mudbutton Clanger                        0.406593
Name: Delver of Secrets // Insectile Aberration, dtype: float64

In [51]:
rec_df['Grizzly Bears'].sort_values()[1:11] # vanilla creature

name
Runeclaw Bear      0.036857
Balduvian Bears    0.081305
Forest Bear        0.081305
Bear Cub           0.081305
Alpine Grizzly     0.099861
Cylian Elf         0.111982
Gnarled Mass       0.119949
Trained Armodon    0.119949
Norwood Ranger     0.124060
Elvish Warrior     0.125622
Name: Grizzly Bears, dtype: float64

In [52]:
rec_df['Oko, Thief of Crowns'].sort_values()[1:11]

name
Bake into a Pie                  0.446577
Shrewd Negotiation               0.450360
Fell the Pheasant                0.485978
Bartered Cow                     0.495809
Savvy Hunter                     0.502182
Wolf's Quarry                    0.513623
Tempting Witch                   0.520718
Fierce Witchstalker              0.523640
Fortifying Provisions            0.534722
Daretti, Ingenious Iconoclast    0.542370
Name: Oko, Thief of Crowns, dtype: float64

In [53]:
rec_df['Gaze of Granite'].sort_values()[1:11]

name
Pernicious Deed      0.203795
Forced March         0.216514
Meltdown             0.231079
Displacement Wave    0.293790
Hammer Mage          0.382592
Dominate             0.401850
Granulate            0.416705
Villainous Wealth    0.441783
Disembowel           0.445213
Stir the Grave       0.472106
Name: Gaze of Granite, dtype: float64

In [54]:
rec_df['Tarmogoyf'].sort_values()[1:11]

name
Lhurgoyf                0.150254
Swarm of Rats           0.359281
Coiling Woodworm        0.366362
Wilderness Elemental    0.371021
Yavimaya Kavu           0.389433
Treefolk Seedlings      0.409718
Dakmor Sorceress        0.410059
People of the Woods     0.410877
Shambling Suit          0.411738
Sylvan Yeti             0.420142
Name: Tarmogoyf, dtype: float64

In [55]:
rec_df['Jace, Vryn\'s Prodigy // Jace, Telepath Unbound'].sort_values()[1:11]

name
Sins of the Past             0.450514
Chandra, Flame's Catalyst    0.466957
Finale of Promise            0.492082
Dreadhorde Arcanist          0.493317
Torrential Gearhulk          0.508705
Jaya Ballard                 0.511588
Diluvian Primordial          0.515140
Dire Fleet Daredevil         0.532010
Obsessive Stitcher           0.536386
Kess, Dissident Mage         0.537863
Name: Jace, Vryn's Prodigy // Jace, Telepath Unbound, dtype: float64

In [56]:
rec_df['Fatal Push'].sort_values()[1:11]

name
Renegade Rallier         0.514177
Fragmentize              0.522034
Wretched Banquet         0.553212
Threads of Disloyalty    0.556422
Overload                 0.557022
Eliminate                0.557502
Thoughtbind              0.559044
Granulate                0.561442
Easy Prey                0.562523
Despark                  0.566456
Name: Fatal Push, dtype: float64

In [57]:
rec_df['Veil of Summer'].sort_values()[1:11]

name
Autumn's Veil           0.304140
Display of Dominance    0.354629
Lazotep Plating         0.443815
Veilstone Amulet        0.460386
Spellbane Centaur       0.473861
Blinding Fog            0.489264
Glaring Spotlight       0.519853
Join Shields            0.520297
Skyshroud Blessing      0.521521
Tortoise Formation      0.522404
Name: Veil of Summer, dtype: float64

In [58]:
rec_df['Urza, Lord High Artificer'].sort_values()[1:11]

name
Oracle's Vault         0.494756
Mind's Desire          0.506143
Aerial Caravan         0.520275
Abbot of Keral Keep    0.533529
Karn, Scion of Urza    0.534546
Temporal Aperture      0.540076
Outpost Siege          0.564563
Djinn of Wishes        0.572008
Vital Splicer          0.572793
Wing Splicer           0.574788
Name: Urza, Lord High Artificer, dtype: float64

In [59]:
rec_df['Underworld Breach'].sort_values()[1:11]

name
Glimpse of Freedom       0.439370
Sweet Oblivion           0.443673
Ox of Agonas             0.449464
Fruit of Tizerus         0.449653
Ichorid                  0.452498
Mogis's Favor            0.463322
Escape Velocity          0.470587
Underworld Charger       0.471657
Loathsome Chimera        0.473991
Underworld Rage-Hound    0.474608
Name: Underworld Breach, dtype: float64

In [60]:
rec_df['Lurrus of the Dream-Den'].sort_values()[1:11]

name
Jegantha, the Wellspring    0.387338
Zirda, the Dawnwaker        0.400557
Kaheera, the Orphanguard    0.413071
Keruga, the Macrosage       0.414855
Gyruda, Doom of Depths      0.422284
Umori, the Collector        0.431399
Obosh, the Preypiercer      0.436415
Lutri, the Spellchaser      0.459570
Isochron Scepter            0.507978
Yorion, Sky Nomad           0.515027
Name: Lurrus of the Dream-Den, dtype: float64

In [76]:
rec_df['Mangara, the Diplomat'].sort_values()[1:11]

name
Ever-Watching Threshold    0.328
Kraum, Ludevic's Opus      0.430
Insight                    0.453
Historian of Zhalfir       0.465
Ghostly Pilferer           0.467
Geth's Grimoire            0.497
Runic Armasaur             0.503
Reparations                0.506
Alms Collector             0.507
Military Intelligence      0.511
Name: Mangara, the Diplomat, dtype: float64

In [61]:
converted_df = converted_df.round(decimals=3)

In [62]:
converted_df.head()

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+2 until,+0/+2 until end,+0/+2 until end of,+0/+2 until end of turn,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 counters,...,legalities_historic,legalities_standard,legalities_pioneer,legalities_duel,legalities_brawl,legalities_penny,rarity_common,rarity_uncommon,rarity_rare,rarity_mythic,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,-0.170
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,-1.299
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,-0.734
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.902,0.0,-0.426,0.0,0.0,0.0,-0.734
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.902,0.0,-1.008,0.0,0.0,0.0,-0.734


In [63]:
rec_df = rec_df.round(decimals=3)

In [64]:
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Stinging Lionfish,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000,0.871,0.922,0.901,0.902,0.773,0.902,0.901,0.936,0.827,0.877,0.843,0.913,0.837,0.908,0.919,0.898,...,0.946,0.751,0.854,0.915,0.874,0.913,0.947,0.930,0.821,0.847,0.928,0.928,0.804,0.934,0.923,0.902,0.857
Sensory Deprivation,0.871,0.000,0.922,0.740,0.731,0.743,0.879,0.704,0.786,0.768,0.778,0.509,0.835,0.916,0.902,0.910,0.785,...,0.915,0.800,0.746,0.874,0.901,0.789,0.901,0.816,0.887,0.912,0.939,0.887,0.631,0.840,0.899,0.879,0.714
Road of Return,0.922,0.922,0.000,0.955,0.883,0.887,0.882,0.907,0.883,0.865,0.870,0.945,0.945,0.867,0.754,0.866,0.842,...,0.838,0.886,0.867,0.899,0.817,0.916,0.774,0.903,0.903,0.899,0.890,0.685,0.881,0.903,0.926,0.830,0.919
Storm Crow,0.901,0.740,0.955,0.000,0.769,0.856,0.916,0.830,0.819,0.799,0.830,0.850,0.885,0.901,0.918,0.867,0.861,...,0.940,0.744,0.834,0.919,0.934,0.857,0.925,0.925,0.920,0.942,0.941,0.938,0.727,0.902,0.933,0.937,0.754
Walking Sponge,0.902,0.731,0.883,0.769,0.000,0.855,0.865,0.472,0.795,0.642,0.800,0.844,0.828,0.894,0.786,0.764,0.852,...,0.874,0.860,0.721,0.905,0.868,0.817,0.895,0.820,0.919,0.942,0.888,0.906,0.694,0.766,0.919,0.764,0.766


In [65]:
# save out the csv
converted_df.to_csv('../Data/converted_df.csv')

In [66]:
# save out the csv
rec_df.to_csv('../Data/recommender_df.csv')

In [67]:
# this is to make a table to merge with the rec_df for filtering.

drop_list = list(vt_list) + ['rarity_common', 'rarity_uncommon', 'rarity_rare', 'rarity_mythic',
                            'activated_ability', 'triggered_ability', 'cmc', 'power', 'power_back', 'toughness',
                            'toughness_back', 'loyalty', 'loyalty_back', 'super_type_Basic', 'super_type_World',
                            'super_type_Legendary', 'super_type_Snow']
filter_df = converted_df.drop(columns=drop_list)
filter_df['cmc'] = df['cmc']
filter_df['card_link'] = df['scryfall_uri']
filter_df.head()

,colors_B,color_identity_B,colors_G,color_identity_G,colors_R,color_identity_R,colors_U,color_identity_U,colors_W,color_identity_W,card_type_Creature,card_type_Instant,card_type_Enchantment,card_type_Sorcery,card_type_Artifact,card_type_Land,card_type_Planeswalker,...,sub_type_Forest,sub_type_Monkey,legalities_legacy,legalities_pauper,legalities_oldschool,legalities_modern,legalities_future,legalities_commander,legalities_vintage,legalities_historic,legalities_standard,legalities_pioneer,legalities_duel,legalities_brawl,legalities_penny,cmc,card_link
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,https://scryfall.com/card/7ed/319/static-orb?u...
Sensory Deprivation,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,https://scryfall.com/card/m14/71/sensory-depri...
Road of Return,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,https://scryfall.com/card/c19/34/road-of-retur...
Storm Crow,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,https://scryfall.com/card/9ed/100/storm-crow?u...
Walking Sponge,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,https://scryfall.com/card/ulg/47/walking-spong...


In [68]:
# merge the two dfs and pickle out the result
filter_rec_df = pd.concat([rec_df, filter_df], axis=1)
filter_rec_df.to_pickle('../Data/filter_rec_df.pkl')

___
### SQL

In [69]:
# create a database
conn = sqlite3.connect('../Data/MTG_Recommender.db')

In [70]:
# create an engine for the database
engine = create_engine('sqlite:///../Data/MTG_Recommender.db')

In [71]:
# save the recommender table to the database
# rec_df.to_sql('recommender', con=engine)
# after doing some outside research on SQL best practices it is not recommended to have such large tables. In fact
# the max limit for columns in SQLite is 2,000 which is much smaller than our 19,267 X 19,267

In [72]:
# save the filter table to the database (minus the card_link column)
filter_df.drop(columns='card_link').to_sql('filter', con=engine, if_exists='replace')

In [73]:
# save the image link table
filter_df[['card_link']].to_sql('links', con=engine, if_exists='replace')

Maybe, from here I can apply user-created filters to the filter_table using SQL queries in order to return a list of cards that fit those filters. Then, from that list pick the top cards whose cosine similarity is lowest compared to the user-selected card.

In [74]:
# proof of concept
filtered_list = ['Static Orb', 'Waterknot', 'Palinchron'] # this list would be returned from the user's filter request

In [75]:
users_card = 'Storm Crow' # an example of a card a user would enter
rec_df[filtered_list].T[users_card].sort_values()[0:11]

name
Waterknot     0.850
Static Orb    0.901
Palinchron    0.901
Name: Storm Crow, dtype: float64

This will be covered in the next notebook. 04-Filter